![Header Image](https://cdn.iconscout.com/icon/free/png-256/python-12-555278.png "header")

# Peer-graded Assignment: Segmenting and Clustering Neighbourhoods in Toronto

---
## Part 1: Creating the dataframe

In [78]:
import pandas as pd

# Set dataframe display size
desired_width = 320

pd.set_option('display.width', desired_width)
pd.set_option('display.max_columns', 10)

# Read table directly from Wikipedia using pandas inbuilt function
df_torontoll = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [79]:
# Keep only Postal Codes with assigned Boroughs
df_torontoll = df_torontoll[df_torontoll.Borough != 'Not assigned']

In [80]:
# Reset the index of the data frame
df_torontoll.index = pd.RangeIndex(len(df_torontoll.index))

# Check for duplicates
number_duplicates = df_torontoll.duplicated(subset='Postal Code', keep='first').sum()
# Check for Borough with Neighbourhood Not assigned
number_not_assigned_neighbourhood = df_torontoll[df_torontoll.Neighbourhood == 'Not assigned'].shape[0]
print("There are {} duplicate Postal Codes in the dataframe and {} rows with no assigned Neighbourhood.".format(number_duplicates, number_not_assigned_neighbourhood))

There are 0 duplicate Postal Codes in the dataframe and 0 rows with no assigned Neighbourhood.


In [81]:
# Check result
df_torontoll.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [82]:
# Check shape
df_torontoll.shape

(103, 3)

---
## Part 2: Adding geographic coordinates to dataframe

In [83]:
# import geocoder

# # with geocoder: worked not even once
# for index, postal_code in zip(df.index, df['Postal Code']):
#     # initialize your variable to None
#     lat_lng_coords = None
#
#     # loop until you get the coordinates
#     while lat_lng_coords is None:
#         g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#         lat_lng_coords = g.latlng
#
#     latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]
#     df.loc[index, 'Latitude'] = latitude
#     df.loc[index, 'Longitude'] = longitude

# df_torontoll.head(10)

In [84]:
# with csv file

df_latlong = pd.read_csv("http://cocl.us/Geospatial_data")
print(df_latlong.head())
for index, postal_code in zip(df_torontoll.index, df_torontoll['Postal Code']):

    lat_lng_coords = df_latlong.loc[df_latlong['Postal Code'] == postal_code, ['Latitude', 'Longitude']]
    df_torontoll.loc[index, 'Latitude'] = lat_lng_coords.iloc[0, 0]
    df_torontoll.loc[index, 'Longitude'] = lat_lng_coords.iloc[0, 1]

df_torontoll.head(10)

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


---
## Part 3: Exploring and clustering the neighbourhoods in Toronto

In [85]:
# Installing libraries
# !conda install -c conda-forge geopy --yes
# !conda install -c conda-forge folium=0.5.0 --yes
# !conda install -c conda-forge folium --yes
# !python3 -m pip install folium

In [86]:
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
    
# tranforming json file into a pandas dataframe library
# from pandas import json_normalize
import json
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [87]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: ____________
CLIENT_SECRET:__________


In [88]:
# The code was removed by Watson Studio for sharing.

In [89]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [90]:
#create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df_torontoll['Latitude'], df_torontoll['Longitude'], df_torontoll['Borough'], df_torontoll['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [91]:
#select all boroughs that contain the word Toronto
df_bor_toronto=df_torontoll[df_torontoll['Borough'].str.contains('Toronto')].reset_index(drop=True)
print(df_bor_toronto.shape)
df_bor_toronto.head(10)

(39, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [92]:
#explore the 5th row neighbourhood - The Beaches
neighbourhood_latitude = df_bor_toronto.loc[4, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = df_bor_toronto.loc[4, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = df_bor_toronto.loc[4, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [93]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=RZDL4SCYCPSJIMMBJHVHJ3JDWLBAHZ05YMDHQQMF5PWG33WZ&client_secret=H3ZL3A4FYGL1OU1C423JRMHNO0O5WTVXVFGWJM2OZBAQMH0X&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [94]:
results = requests.get(url).json()

In [95]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [96]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [97]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [98]:
# explore all neighbourhoods in boroughs that contain the word "Toronto"


# fuction to repeat the same process as with The Beaches to all neighbourhoods in boroughs that contain the word "Toronto"
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [99]:
borough_Toronto_venues = getNearbyVenues(names=df_bor_toronto['Neighbourhood'],
                                   latitudes=df_bor_toronto['Latitude'],
                                   longitudes=df_bor_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [100]:
print(borough_Toronto_venues.shape) #the size of the resulting dataframe
borough_Toronto_venues.head()

(1638, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [101]:
# check how many venues were returned for each neighbourhood
borough_Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,67,67,67,67,67,67
Christie,17,17,17,17,17,17
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


In [102]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(borough_Toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


In [103]:
# analyse each neighbourhood in boroughs that contain the word "Toronto"

# one hot encoding
borough_Toronto_onehot = pd.get_dummies(borough_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
borough_Toronto_onehot['Neighbourhood'] = borough_Toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [borough_Toronto_onehot.columns[-1]] + list(borough_Toronto_onehot.columns[:-1])
borough_Toronto_onehot = borough_Toronto_onehot[fixed_columns]

borough_Toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,...,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,...,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,...,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,...,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,...,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,...,0,0,0,0,0


In [104]:
borough_Toronto_onehot.shape

(1638, 235)

In [105]:
# let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

borough_Toronto_grouped = borough_Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
borough_Toronto_grouped.head(10)

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,...,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.0,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.0,0.0,0.055556
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,...,0.0,0.000000,0.0,0.0,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,...,0.0,0.014925,0.0,0.0,0.014925
5,Christie,0.000000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.0,0.0,0.000000
6,Church and Wellesley,0.013333,0.0000,0.0000,0.0000,...,0.0,0.000000,0.0,0.0,0.026667
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,...,0.0,0.010000,0.0,0.0,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.0,0.0,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,...,0.0,0.000000,0.0,0.0,0.000000


In [106]:
borough_Toronto_grouped.shape

(39, 235)

In [107]:
# print each neighbourhood along with the top 5 most common venues
num_top_venues = 5

for hood in borough_Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = borough_Toronto_grouped[borough_Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.10
1        Beer Bar  0.03
2  Farmers Market  0.03
3            Café  0.03
4     Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3       Burrito Place  0.05
4  Italian Restaurant  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.11
1  Gym / Fitness Center  0.06
2               Brewery  0.06
3                   Spa  0.06
4            Skate Park  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.19
1       Airport Lounge  0.12
2          Coffee Shop  0.06
3  Rental Car Location  0.06
4   Airport Food Court  0.06


----Central Bay Street----
                 v

In [108]:
# turn into a pandas dataframe

# a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [109]:
# create the new dataframe and display the top 10 venues for each neighbourhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = borough_Toronto_grouped['Neighbourhood']

for ind in np.arange(borough_Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(borough_Toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Beer Bar,...,Farmers Market,Restaurant,Café,Cheese Shop,Department Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,...,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Auto Workshop,Park,Comic Shop,...,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Boutique,Rental Car Location,...,Coffee Shop,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport Terminal
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,...,Department Store,Salad Place,Bubble Tea Shop,Burger Joint,Dessert Shop


In [110]:
# Cluster Neighbourhoods
# Run k-means to cluster the neighbourhood into 5 clusters
# set number of clusters
kclusters = 5

borough_Toronto_grouped_clustering = borough_Toronto_grouped.drop('Neighbourhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(borough_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 0,
       0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0], dtype=int32)

In [111]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood

# add clustering labels

neighbourhoods_venues_sorted.insert(0,"Cluster Labels",kmeans.labels_)

In [112]:
neighbourhoods_venues_sorted.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,...,Farmers Market,Restaurant,Café,Cheese Shop,Department Store
1,0,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,...,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
2,0,"Business reply mail Processing Centre, South C...",Light Rail Station,Auto Workshop,Park,...,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Boutique,...,Coffee Shop,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport Terminal
4,0,Central Bay Street,Coffee Shop,Sandwich Place,Café,...,Department Store,Salad Place,Bubble Tea Shop,Burger Joint,Dessert Shop


In [113]:
borough_Toronto_merged = df_bor_toronto.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
borough_Toronto_merged = borough_Toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

borough_Toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,...,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,...,Mexican Restaurant,Japanese Restaurant,Hobby Shop,Fried Chicken Joint,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,...,Cosmetics Shop,Japanese Restaurant,Fast Food Restaurant,Lingerie Store,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,...,Cocktail Bar,Cosmetics Shop,Moroccan Restaurant,Breakfast Spot,Park
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,...,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio


In [114]:
# visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(borough_Toronto_merged['Latitude'], borough_Toronto_merged['Longitude'], borough_Toronto_merged['Neighbourhood'], borough_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine each cluster

In [115]:
#CLUSTER 1

borough_Toronto_merged.loc[borough_Toronto_merged['Cluster Labels'] == 0, borough_Toronto_merged.columns[[1] + list(range(5, borough_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Pub,...,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
1,Downtown Toronto,0,Coffee Shop,Diner,Yoga Studio,...,Mexican Restaurant,Japanese Restaurant,Hobby Shop,Fried Chicken Joint,Distribution Center
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,...,Cosmetics Shop,Japanese Restaurant,Fast Food Restaurant,Lingerie Store,Bookstore
3,Downtown Toronto,0,Restaurant,Café,Coffee Shop,...,Cocktail Bar,Cosmetics Shop,Moroccan Restaurant,Breakfast Spot,Park
5,Downtown Toronto,0,Coffee Shop,Seafood Restaurant,Cocktail Bar,...,Farmers Market,Restaurant,Café,Cheese Shop,Department Store
6,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,...,Department Store,Salad Place,Bubble Tea Shop,Burger Joint,Dessert Shop
7,Downtown Toronto,0,Grocery Store,Café,Park,...,Restaurant,Italian Restaurant,Athletics & Sports,Diner,Nightclub
8,Downtown Toronto,0,Coffee Shop,Café,Restaurant,...,Hotel,Thai Restaurant,Steakhouse,Concert Hall,Cosmetics Shop
9,West Toronto,0,Bakery,Pharmacy,Middle Eastern Restaurant,...,Bar,Café,Music Venue,Pet Store,Brewery
10,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,...,Scenic Lookout,Restaurant,Brewery,History Museum,Baseball Stadium


In [116]:
#CLUSTER 2

borough_Toronto_merged.loc[borough_Toronto_merged['Cluster Labels'] == 1, borough_Toronto_merged.columns[[1] + list(range(5, borough_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,1,Garden,Yoga Studio,Department Store,...,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [117]:
#CLUSTER 3

borough_Toronto_merged.loc[borough_Toronto_merged['Cluster Labels'] == 2, borough_Toronto_merged.columns[[1] + list(range(5, borough_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,2,Gym,Restaurant,Deli / Bodega,...,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [118]:
#CLUSTER 4

borough_Toronto_merged.loc[borough_Toronto_merged['Cluster Labels'] == 3, borough_Toronto_merged.columns[[1] + list(range(5, borough_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Park,Swim School,Bus Line,...,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
33,Downtown Toronto,3,Park,Playground,Trail,...,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [119]:
#CLUSTER 5

borough_Toronto_merged.loc[borough_Toronto_merged['Cluster Labels'] == 4, borough_Toronto_merged.columns[[1] + list(range(5, borough_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,4,Trail,Neighborhood,Pub,...,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio


## The END

### Thank you